In [1]:
import os
os.chdir(os.path.join(os.path.abspath(os.path.expanduser("~")), "Documents", "Personal", "Test"))
from scripts.tools.issuetracker import IssuetrackerAPI as _ISAPI, Issue
import pickle

import datetime, dateutil
from_date = datetime.datetime(2016, 6, 1, tzinfo=dateutil.tz.tzutc())
today =datetime.date.today()
to_date = datetime.datetime(today.year, today.month, today.day, tzinfo=dateutil.tz.tzutc())


class IssuetrackerAPI(_ISAPI):
    
    def make_pickle_cache(self, *args):
        issues = super().download_issues(*args)
        with open("gantt_dummy.pkl", 'wb') as f:
            pickle.dump(issues, f)
        return issues
    def download_issues(self, *args):
        if not os.path.exists("gantt_dummy.pkl"):
            return self.make_pickle_cache(*args)
        else:
            with open("gantt_dummy.pkl", 'rb') as f:
                try:
                    issues = pickle.load(f)
                except (pickle.PickleError, OSError, EOFError):
                    pass
                else:
                    return issues
            return self.make_pickle_cache(*args)
        return issues


In [2]:
from officelib import wordlib
from scripts.tools.issuetracker.frs_update import main
import shutil
word = wordlib.Word()
pdfpth = "pdfs6"
try:
    word.ActiveDocument.Close(False)
except:
    pass
try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', "2016-6-1", 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038B2 Rev A Functional Requirements Specification Updates")
#os.startfile('"%s"'%res)
print()
print("Done")

Begin Main

Logging into Issuetracker...Downloading projects...

Got update: PGS_BEGIN_PROCESS_PDFS 
Creating PDF Index...
Creating Word Document...
Creating index 1 of 34
Creating index 6 of 34                    
Creating index 9 of 34                    
Creating index 16 of 34                    
Creating index 22 of 34                    
Creating index 29 of 34                    
Creating index 34 of 34                    
Got update: PGS_IPG_FLAG_WAITING 
Got update: MPD_ALL_COMPLETE 
Got update: PGS_PDFTK_INFO_BEGIN 
Getting PDF Page Info (29/34)
Got update: PGS_IPG_FLAG_WAITING 2
Getting PDF Page Info (34/34)
Got update: PGS_PDFTK_INFO_DONE 
Got update: PGS_PROCESS_PDFS_DONE 
Got update: PGS_PREPARE_POSTSCRIPT 
Got update: PGS_BEGIN_BIND_PDF 
Got update: PGS_DONE 
Done


In [3]:
# 3/2/2017
from officelib import wordlib
from scripts.tools.issuetracker.frs_update import main
import shutil
word = wordlib.Word()
pdfpth = "Software_FRS_170302"
try:
    word.ActiveDocument.Close(False)
except:
    pass
try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', None, 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038B2 Rev A Functional Requirements Specification")
#os.startfile('"%s"'%res)
print()
print("Done")

Begin Main

Logging into Issuetracker...Downloading projects...

Got update: PGS_BEGIN_PROCESS_PDFS 
Creating PDF Index...
Creating Word Document...
Creating index 17 of 86                    
Creating index 48 of 86                    
Creating index 81 of 86                    
Creating index 86 of 86                    
Got update: PGS_IPG_FLAG_WAITING 
Got update: PGS_IPG_FLAG_WAITING 2
Got update: MPD_ALL_COMPLETE 
Got update: PGS_PDFTK_INFO_BEGIN 
Getting PDF Page Info (86/86)                    
Got update: PGS_PDFTK_INFO_DONE 
Got update: PGS_PROCESS_PDFS_DONE 
Got update: PGS_PREPARE_POSTSCRIPT 
Got update: PGS_BEGIN_BIND_PDF 
Got update: PGS_DONE 
Done
